# Using CSBM generated data to train a model

In [1]:
import torch
import matplotlib.pyplot as plt

In [2]:
example_dataset = torch.load('./csbm_base/base_0.pt')
T = example_dataset.t.unique().size()[0]
n = example_dataset.x.size()[0]
c = example_dataset.y.unique().size()[0]
d = example_dataset.x.size()[1]

In [3]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(d, 128)
        self.conv2 = GCNConv(128, c)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

## Retrain model for each task

In [4]:
from measures import Result

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.set_printoptions(precision=2)

tasks = [example_dataset.t == t for t in range(T)]

for i in range(10):
    base_data = torch.load(f'./csbm_base/base_{i}.pt')
    zero_data = torch.load(f'./csbm_zero/zero_{i}.pt')
    feat_data = torch.load(f'./csbm_feat/feat_{i}.pt')
    struct_data = torch.load(f'./csbm_hom/hom_{i}.pt')
    homophily_data = torch.load(f'./csbm_struct/struct_{i}.pt')
    class_data = torch.load(f'./csbm_class/class_{i}.pt')

    
    base_result = Result(base_data, GCN().to(device), [base_data.t == t for t in range(T)])
    zero_result = Result(zero_data, GCN().to(device), [zero_data.t == 0])
    feat_result = Result(feat_data, GCN().to(device), [feat_data.t == t for t in range(T)])
    struct_result = Result(struct_data, GCN().to(device), [struct_data.t == t for t in range(T)])
    homophily_result = Result(homophily_data, GCN().to(device), [homophily_data.t == t for t in range(T)])
    class_result = Result(class_data, GCN().to(device), [class_data.t == t for t in range(T)])
    
    results = [base_result, zero_result, feat_result, struct_result, homophily_result, class_result]
    names = {base_result: 'Base-CSBM',
             zero_result: 'Zero-CSBM',
             feat_result: 'Feature-CSBM',
             struct_result: 'Structure-CSBM',
             homophily_result: 'Homophily-CSBM',
             class_result: 'Class-CSBM'}
    for result in results:
        result.learn()
    
    for result in results:
        print('\n' + 10 * '=' + f' {names[result]} ' + 10 * '=')
        print(result.get_result_matrix())
        ap = result.get_average_accuracy()
        af = 0 if result == zero_result else result.get_average_forgetting_measure()
        print(f'\n AP: {ap:.2f}'.ljust(10) + f' AF: {af:.2f}')

cuda

========== Base-CSBM ==========
tensor([[0.40, 0.20, 0.15, 0.40, 0.35, 0.15, 0.25, 0.40, 0.25, 0.45],
        [0.25, 0.30, 0.20, 0.30, 0.40, 0.40, 0.45, 0.35, 0.35, 0.50],
        [0.40, 0.45, 0.30, 0.20, 0.45, 0.30, 0.20, 0.50, 0.50, 0.35],
        [0.30, 0.40, 0.35, 0.30, 0.40, 0.20, 0.15, 0.10, 0.60, 0.40],
        [0.40, 0.30, 0.45, 0.40, 0.60, 0.50, 0.50, 0.35, 0.40, 0.60],
        [0.35, 0.35, 0.30, 0.25, 0.65, 0.60, 0.40, 0.35, 0.50, 0.40],
        [0.35, 0.30, 0.35, 0.35, 0.45, 0.45, 0.25, 0.45, 0.55, 0.45],
        [0.65, 0.50, 0.50, 0.40, 0.45, 0.35, 0.30, 0.35, 0.45, 0.65],
        [0.45, 0.55, 0.35, 0.25, 0.60, 0.35, 0.45, 0.30, 0.55, 0.60],
        [0.35, 0.40, 0.30, 0.40, 0.55, 0.35, 0.30, 0.30, 0.50, 0.80]])

 AP: 0.43 AF: 0.15

========== Zero-CSBM ==========
tensor([[0.29]])

 AP: 0.29 AF: 0.00

========== Feature-CSBM ==========
tensor([[0.15, 0.25, 0.25, 0.35, 0.35, 0.35, 0.20, 0.20, 0.15, 0.10],
        [0.25, 0.30, 0.25, 0.30, 0.40, 0.30, 0.25, 0.30, 0.15, 0.